In [3]:
# Choose the stock symbol

stock = 'AAPL'

In [4]:
import good_morning as gm
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
import numpy as np


In [5]:
def get_Total_Equity(Stock):

    kr = gm.FinancialsDownloader()
    kr_frames = kr.download(Stock)
    S0 = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Total stockholders\' equity')].iloc[0,-1]
    
    return S0


def get_Volatility(Stock, delta_time):
    url = "https://app.quotemedia.com/quotetools/getHistoryDownload.csv?&webmasterId=501&startDay=02&startMonth=02&startYear=2002&endDay=22&endMonth=09&endYear=2017&isRanged=false&symbol={}".format(Stock)
    
    data = pd.read_csv(url, parse_dates = True, index_col = 0)
  
    data = data.loc[:,('close')]
    data = data.sort_index(ascending = False)
    
    volatility = np.std(data.tail(delta_time))
    
    return volatility

def get_Maturity(Stock):
    kr = gm.FinancialsDownloader()
    kr_frames = kr.download(Stock)
    current_liability = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Current liabilities')].iloc[0,-1]
    non_current_liability = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Non-current liabilities')].iloc[0,-1]
    time = (current_liability*0.6 + non_current_liability*2)/(current_liability + non_current_liability)
    
    return time

def get_Debt_Face_Value(Stock,risk_free):
    
    kr = gm.FinancialsDownloader()
    kr_frames = kr.download(Stock)
    current_liability = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Current liabilities')].iloc[0,-1]
    non_current_liability = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Non-current liabilities')].iloc[0,-1]
    
    debt1 = current_liability**(1+risk_free/200)
    debt2 = non_current_liability**(1+risk_free*2/100)
    debt = debt1 + debt2
    debt = 1
    
    return debt

def merton_solve(param):
    
    V0 = param[0]         #initial value for V0
    sigmaV = param[1]       # inicial value for volatility
    
    
    d1 = (np.log(V0/B) + (r+(sigmaV**2)/2)*T)/(sigmaV*(T**(1/2)))
    d2 = d1 - sigmaV*(T**(1/2))
    
    F = V0*norm.cdf(d1) - B*np.exp(-r*T)*norm.cdf(d2) - S0
    G = norm.cdf(d1)*sigmaV*V0 - sigmaS*S0
    
    return F**2 + G**2      # We want to minimize this function


In [6]:
S0 =   get_Total_Equity(stock)     #Equity at time 0
sigmaS = get_Volatility(stock, 50)    # Instant volatility
r = 0.05       # risk-free rate
T = get_Maturity(stock)          #Maturity
B = get_Debt_Face_Value(stock,5)         #Face value of debt obligation

# Now we construct an optimization function to find the real values of S0 and 
# SigmaV

res = minimize(merton_solve, (13,0.5))

# Now we compute d1 and d2 explicity
d1 = (np.log(res['x'][0]/B) + (r+res['x'][1]**2/2)*T)/(res['x'][1]*(T**(1/2)))
d2 = d1 - res['x'][1]*(T**(1/2))

# The probability of default is


probability = norm.cdf(-d2)

               open      high       low    close    volume  changed changep  \
date                                                                          
2017-10-20  156.610  157.7500  155.9600  156.250  23974146    0.270   0.17%   
2017-10-19  156.750  157.0800  155.0200  155.980  42584166   -3.780  -2.37%   
2017-10-18  160.420  160.7100  159.6000  159.760  16374164   -0.710  -0.44%   
2017-10-17  159.780  160.8700  159.2300  160.470  18997275    0.590   0.37%   
2017-10-16  157.900  160.0000  157.6500  159.880  24121452    2.890   1.84%   
2017-10-13  156.730  157.2800  156.4100  156.990  16394188    0.990   0.63%   
2017-10-12  156.350  157.3700  155.7299  156.000  16125054   -0.550  -0.35%   
2017-10-11  155.970  156.9800  155.7500  156.550  16905640    0.650   0.42%   
2017-10-10  156.055  158.0000  155.1000  155.900  15617014    0.060   0.04%   
2017-10-09  155.810  156.7300  155.4850  155.840  16262923    0.540   0.35%   
2017-10-06  154.970  155.4900  154.5600  155.300  17

In [8]:
print('The probability of default of {} is : {}'.format(stock,probability))

The probability of default of AAPL is : 1.95032159621634e-05
